## Completar Página 12

En esta notebook arreglamos los datos de Página12 que están mal bajados

In [1]:
import pandas as pd

df = pd.read_excel("../data/pagina_12.xlsx", header=0)

print(f"Tenemos {df.shape[0]} noticias")

Tenemos 810 noticias


In [35]:
df["titulo_arreglado"] = df["titulo"]

In [2]:
df.iloc[0]

medio                                             pagina 12
url       https://elplanetaurbano.com/2019/09/top-5-dest...
fecha                                   2019-09-21 10:00:29
titulo                            TOP 5 DESTINOS CANNÁBICOS
Name: 0, dtype: object

Busquemos los índices incompletos

In [39]:
from tqdm.auto import tqdm

incomplete_idx = []

for idx, row in tqdm(df.iterrows(), total=len(df)):
    title = row["titulo"]
    if "..." in title:
        incomplete_idx.append(idx)

In [40]:
len(incomplete_idx)

368

Bajemos las noticias


In [5]:
from newspaper import Article

articles = {}

for idx in tqdm(incomplete_idx):
    row = df.loc[idx]
    article = Article(row.url)
    article.download()
    article.parse()
    
    articles[idx] = article

In [6]:
art = articles[3]

In [13]:
type(art.clean_doc)

lxml.html.HtmlElement

In [15]:
doc = art.doc

doc

<Element html at 0x7efc6589a4f0>

Lo meto en otra columna

In [36]:
for idx in articles:
    art = articles[idx]
    title_meta = art.doc.cssselect('meta[property="og:title"]')[0]
    fixed_title = title_meta.get('content')
    
    df.loc[idx, "titulo_arreglado"] = fixed_title

Lo salvamos!

In [42]:
df.to_csv("../data/pagina_12_titulos_arreglados.csv")

## Corregir Excel original

In [45]:

todas_las_noticias = pd.read_excel("../data/TITULOS_SENTIMENT ANALYSIS.xlsx", header=0, names=["medio", "fecha", "titulo"])

todas_las_noticias.medio.value_counts()

InfoBAE                5956
La Nación              4863
Clarin                 4112
Ambito Financiero      1739
Cronista               1513
La Izquierda Diario    1327
Perfil                 1036
Página 12               824
Tiempo Argentino        712
El Popular              346
Prensa Obrera           192
Name: medio, dtype: int64

Tenemos 824 noticias de P12 -- en el anterior teníamos 810

In [62]:
p12 = todas_las_noticias[todas_las_noticias["medio"] == "Página 12"]


for idx, row in p12.iterrows():
    if "..." in row["titulo"]:
        try:
            titulo_arreglado = df[df["titulo"] == row["titulo"]].iloc[0]["titulo_arreglado"]
            
            todas_las_noticias.loc[idx, "titulo"] = titulo_arreglado
            #print(titulo_arreglado)
        except IndexError:
            print(f"No pudimos encontrar {row['titulo']}")


No pudimos encontrar Alberto Fernández en Salta: "Voy a gobernar una Arg...
No pudimos encontrar Macri en Belgrano con guión estricto: distribuyó le...
No pudimos encontrar Alberto Fernández con Beatriz Sarlo: "Cristina tuvo...
No pudimos encontrar Nuevo DNU de Macri: "Lo van a declarar inconstituci...
No pudimos encontrar Macri en Santa Fe: "Todos tuvimos que poner el homb...
No pudimos encontrar Cornejo: "¿Qué puta tienen que venir Macri o Albert...
No pudimos encontrar Macri contra el aborto: "Claramente a favor de las ...
No pudimos encontrar Alberto Fernández le respondió a Macri por su tuit ...
No pudimos encontrar Alberto Fernández: "No podemos vivir en paz con est...
No pudimos encontrar Macri, de besar un pie a decir que hay "una pata so...
No pudimos encontrar La "Ley mentira" que el macrismo impulsa en plena c...
No pudimos encontrar Marocco: “Alberto Fernández en Salta acompaña al Fr...
No pudimos encontrar Alberto Fernández: "Macri deja un escenario similar...
No pudimos e

In [64]:
todas_las_noticias.to_csv("../data/titulos.csv")

In [66]:
todas_las_noticias["medio"].value_counts()

InfoBAE                5956
La Nación              4863
Clarin                 4112
Ambito Financiero      1739
Cronista               1513
La Izquierda Diario    1327
Perfil                 1036
Página 12               824
Tiempo Argentino        712
El Popular              346
Prensa Obrera           192
Name: medio, dtype: int64

In [67]:


incomplete_idx = []
new_p12 = todas_las_noticias[todas_las_noticias["medio"] == "Página 12"]
for idx, row in new_p12.iterrows():
    if 